In [ ]:
import os
import nltk
import plyj.parser as plyj
from nltk.tokenize import word_tokenize
import numpy as np

nltk.download('punkt')

In [ ]:
# Función para tokenizar un archivo
def tokenizar_archivo(archivo):
    with open(archivo, 'r') as file:
        codigo = file.read()
        tokens = word_tokenize(codigo)
    return tokens

# Función para parsear el código y contar características
def parsear_y_contar_caracteristicas(codigo):
    parser = plyj.Parser()
    tree = parser.parse_string(codigo)
    tokens = [str(token).strip() for token in tree.iter_tokens()]
    features = {token: tokens.count(token) for token in set(tokens)}
    return features

# Carpeta donde se encuentran los códigos
carpeta_codigos = ".\\conplag\\programs"

# Leer todos los códigos en la carpeta
codigos_tokenizados = {}  # Usamos un diccionario para almacenar los tokens por nombre de archivo
for filename in os.listdir(carpeta_codigos):
    filepath = os.path.join(carpeta_codigos, filename)
    if os.path.isfile(filepath):
        tokens = tokenizar_archivo(filepath)
        codigos_tokenizados[filename] = tokens

# Mostrar los tokens por nombre de archivo
for nombre_archivo, tokens in codigos_tokenizados.items():
    print(f"Tokens del archivo '{nombre_archivo}':\n{tokens}\n")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Convertir todos los códigos tokenizados a texto plano para TF-IDF y Markovify
codigos_planos = [' '.join(codigo) for codigo in codigos_tokenizados.values()]

# Construir un vectorizador TF-IDF
vectorizador_tfidf = TfidfVectorizer()
tfidf_matrix = vectorizador_tfidf.fit_transform(codigos_planos)

# Función para calcular la similitud de coseno entre dos vectores de frecuencia de términos (TF-IDF)
def similitud_coseno(vector1, vector2):
    return cosine_similarity(vector1.reshape(1, -1), vector2.reshape(1, -1))[0][0]

# Función para calcular la similitud de Jaccard entre dos conjuntos de tokens
def similitud_jaccard(tokens1, tokens2):
    set1 = set(tokens1)
    set2 = set(tokens2)
    return len(set1.intersection(set2)) / len(set1.union(set2))

data = []

for nombre_archivo1, codigo_tokenizado1 in codigos_tokenizados.items():
    for nombre_archivo2, codigo_tokenizado2 in codigos_tokenizados.items():
        if nombre_archivo1 != nombre_archivo2:  
            sim_jaccard = similitud_jaccard(codigo_tokenizado1, codigo_tokenizado2)
            if sim_jaccard >= 0.5:
                codigo_plano1 = ' '.join(codigo_tokenizado1)
                codigo_plano2 = ' '.join(codigo_tokenizado2)
                tfidf_codigo1 = vectorizador_tfidf.transform([codigo_plano1])
                tfidf_codigo2 = vectorizador_tfidf.transform([codigo_plano2])
                sim_coseno = similitud_coseno(tfidf_codigo1, tfidf_codigo2)
                # Agregar nombres de archivos a la lista data
                data.append([nombre_archivo1, nombre_archivo2, sim_jaccard, sim_coseno])
                print(f"Comparando código '{nombre_archivo1}' con código '{nombre_archivo2}':")
                print(f"Similitud de coseno: {round(sim_coseno, 4)}")
                print(f"Similitud de Jaccard: {round(sim_jaccard, 4)}\n")

In [ ]:
# Función para determinar el grado de pertenencia al plagio
def grado_pertenencia(sim_jaccard, sim_coseno):
    # Establecer las reglas difusas
    if sim_jaccard >= 0.5 and sim_coseno >= 0.5:
        return "------ ALTA"
    else:
        return None  # Retornar None si el grado de pertenencia es menor a 0.5 en alguna similitud

for nombre_archivo1, nombre_archivo2, sim_jaccard, sim_coseno in data:
    # Computar el grado de pertenencia al plagio
    pertenencia = grado_pertenencia(sim_jaccard, sim_coseno)
    
    # Imprimir el grado de pertenencia al plagio junto con los nombres de los archivos
    if pertenencia is not None:  # Imprimir solo si el grado de pertenencia no es None
        print(f"Comparando código '{nombre_archivo1}' con código '{nombre_archivo2}':")
        print("Similitud Jaccard:", round(sim_jaccard, 4))
        print("Similitud Coseno:", round(sim_coseno, 4))
        print("Grado de pertenencia al plagio:", pertenencia)